# Clasificador Sentimientos

Este dataset contiene informacion de multitud de tweets. Con los datos de estos tweets podemos crear un modelo capaz de predecir el sentimiento de un texto. La columna sentimiento tiene los dos posibles valores, siendo 0 un sentimiento de texto negativo y 1 un sentimiento de texto positivo.

In [1]:
# Importamos las librerias necesarias
import os
import re
import pickle
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...


True

### EDA

In [2]:
# Importamos los datos
df=pd.read_csv("data/tweets.csv", encoding="ISO-8859-1", names=["Sentimiento", "ID", "Fecha", "Tipo", "Usuario", "Contenido"])
df.head()

,Sentimiento,ID,Fecha,Tipo,Usuario,Contenido
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
# Nos quedamos con las columnas necesarias
df=df[["Contenido", "Sentimiento"]]
df.head()

,Contenido,Sentimiento
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [4]:
# Vemos los registros que tenemos
print(f"Numero de registros: {df.shape[0]}")

Numero de registros: 1600000


In [5]:
# Observamos los posibles valores del sentimiento
df["Sentimiento"].value_counts()

Sentimiento
0    800000
4    800000
Name: count, dtype: int64

In [6]:
# Convertimos el 4 en 1
df["Sentimiento"]=df["Sentimiento"].apply(lambda x: 1 if x==4 else x)

In [7]:
# Comprobamos los valores del sentimiento
df["Sentimiento"].value_counts()

Sentimiento
0    800000
1    800000
Name: count, dtype: int64

In [8]:
# Pasamos el centenido a una tupla de listas para su manipulacion
contenido, sentimiento=list(df["Contenido"]), list(df["Sentimiento"])

In [9]:
# Definimos los emeojis y los stopwords para poder analizarlo de manera mas optima
emoticonos={':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
              ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
              ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
              ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
              '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
              '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
              ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

stopwords=['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from', 
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [10]:
# Funcion para procesar los datos del contenido
def procesarContenido(contenido:str, emoticonos:dict, stopwords:list)->str:

    lematizador=WordNetLemmatizer()

    contenido_minus=contenido.lower()

    contenido_url=re.sub(r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)", ' URL', contenido_minus)

    for emoticono in emoticonos.keys():
        
            contenido_emoticono=contenido_url.replace(emoticono, "EMOTICONO" + emoticonos[emoticono])
        
    contenido_usuario=re.sub("@[^\s]+", ' USUARIO', contenido_emoticono)

    contenido_alfanum=re.sub("[^a-zA-Z0-9]", " ", contenido_usuario)

    contenido_secuencia=re.sub(r"(.)\1\1+", r"\1\1", contenido_alfanum)

    palabras_precesadas=[]
    
    for palabra in contenido_secuencia.split():
        
        if len(palabra) > 1 and palabra not in stopwords:
            
            palabra_final=lematizador.lemmatize(palabra)
            palabras_precesadas.append(palabra_final)

    return " ".join(palabras_precesadas)

In [11]:
# Elegimos un contenido aleatorio y lo procesamos
print(contenido[22])
procesarContenido(contenido[22], emoticonos, stopwords)

@angry_barista I baked you a cake but I ated it 


'USUARIO baked cake but ated'

In [12]:
# Realizamos el porcesamiento con todos los contenidos que tenemos
contenido_procesado=list(map(lambda x: procesarContenido(x, emoticonos, stopwords), contenido))
contenido_procesado[1]

'upset update facebook texting might cry result school today also blah'

### Seeleccion del modelo

In [13]:
# Realizamos la separacion de nuestros datos en entrenamiento y test
X_entrenamiento, X_test, y_entrenamiento, y_test=train_test_split(contenido_procesado, sentimiento, test_size=0.05, random_state=0)

In [14]:
# Creamos un objeto vectorizador
vectorizador=TfidfVectorizer(ngram_range=(1,2), max_features=500000)

In [15]:
# Entrenamos el vectorizador
vectorizador.fit(X_entrenamiento)

TfidfVectorizer(max_features=500000, ngram_range=(1, 2))

In [16]:
# Vectorizamos los datos
X_entrenamiento_vectorizado=vectorizador.transform(X_entrenamiento)
X_test_vectorizado=vectorizador.transform(X_test)

In [17]:
# Creamos unos objetos modelo vacios para entrenarlos y ver que buenos son
modelos=[("BNB", BernoulliNB(alpha=2)),
         ("SVC", LinearSVC()),
         ("Logistica", LogisticRegression(C=2, max_iter=1000, n_jobs=-1))]

In [18]:
# Iteramos por los modelos 
for nombre, modelo in modelos:

    # Entrenamos el modelo con los datos de entrenamiento
    modelo.fit(X_entrenamiento_vectorizado, y_entrenamiento)

    # Realizamos las predicciones con el test
    y_prediccion=modelo.predict(X_test_vectorizado)

    print(f"Modelo: {nombre}\n{classification_report(y_test, y_prediccion)}")

Modelo: BNB
              precision    recall  f1-score   support

           0       0.80      0.79      0.80     39989
           1       0.79      0.81      0.80     40011

    accuracy                           0.80     80000
   macro avg       0.80      0.80      0.80     80000
weighted avg       0.80      0.80      0.80     80000

Modelo: SVC
              precision    recall  f1-score   support

           0       0.81      0.79      0.80     39989
           1       0.80      0.81      0.81     40011

    accuracy                           0.80     80000
   macro avg       0.80      0.80      0.80     80000
weighted avg       0.80      0.80      0.80     80000

Modelo: Logistica
              precision    recall  f1-score   support

           0       0.82      0.80      0.81     39989
           1       0.81      0.83      0.82     40011

    accuracy                           0.82     80000
   macro avg       0.82      0.82      0.82     80000
weighted avg       0.82      0.8

### Pipeline

In [19]:
# Realizamos la separacion de nuestros datos en entrenamiento y test nuevamente
X_entrenamiento, X_test, y_entrenamiento, y_test=train_test_split(contenido_procesado, sentimiento, test_size=0.05, random_state=0)

In [20]:
# Creamos un objeto vectorizador
vectorizador=TfidfVectorizer(ngram_range=(1,2), max_features=500000)

In [21]:
# Creamos un objeto del modelo bnb
bnb=BernoulliNB(alpha=2)

In [22]:
# Creamos un pipeline con ambos objetos
pipe=Pipeline([("Vectorizador",vectorizador), ("BNB", bnb)])

In [23]:
# Entrenamos el pipe (el Vectorizador y el BNV)
pipe.fit(X_entrenamiento, y_entrenamiento)

Pipeline(steps=[('Vectorizador',
                 TfidfVectorizer(max_features=500000, ngram_range=(1, 2))),
                ('BNB', BernoulliNB(alpha=2))])

In [24]:
# Realizamos las predicciones con el test
y_prediccion=pipe.predict(X_test)

In [25]:
# Mostramos el reporte
print(classification_report(y_test, y_prediccion))

              precision    recall  f1-score   support

           0       0.80      0.79      0.80     39989
           1       0.79      0.81      0.80     40011

    accuracy                           0.80     80000
   macro avg       0.80      0.80      0.80     80000
weighted avg       0.80      0.80      0.80     80000



### Testeo del modelo

In [26]:
frase="Hello, im happy"
resultado=pipe.predict([frase])
resultado[0]

1

### Almacenado del modelo

In [27]:
# Volcamos el pipe final en un archivo pickle
with open("pipeline.pkl", "wb") as archivo:
    
    pickle.dump(pipe, archivo)

In [28]:
# Comprobamos que existe el archivo
os.path.exists("pipeline.pkl")

True